In [ ]:
# 🛠 Install everything
!pip install pygame moderngl diffusers transformers accelerate safetensors huggingface_hub moviepy imageio pillow --quiet

In [ ]:
# 🖥️ Base imports
import os
import sys
import time
import numpy as np
import torch
import pygame
import imageio
import moderngl
import argparse

from PIL import Image
from diffusers import StableDiffusionPipeline
from moviepy import VideoFileClip, AudioFileClip

In [ ]:
# 🎨 Scene Generator
class SceneGenerator:
    def __init__(self, prompts, output_folder="generated_scenes"):
        self.prompts = prompts
        self.output_folder = output_folder
        os.makedirs(output_folder, exist_ok=True)
        model_id = "CompVis/stable-diffusion-v1-4"
        self.pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float32)
        self.pipe.to("cuda" if torch.cuda.is_available() else "cpu")

    def generate(self):
        scene_paths = []
        for idx, prompt in enumerate(self.prompts):
            output_path = os.path.join(self.output_folder, f"scene_{idx}.png")
            if not os.path.exists(output_path):
                print(f"🎨 Generating scene {idx+1}: {prompt}")
                image = self.pipe(prompt).images[0]
                image.save(output_path)
            scene_paths.append(output_path)
        return scene_paths

In [ ]:
# 🎭 Scene Fader
class SceneFader:
    def __init__(self, ctx, scene_paths):
        self.ctx = ctx
        self.textures = [self.load_texture(p) for p in scene_paths]
        self.current = 0
        self.next = 1
        self.transition = 0.0

    def load_texture(self, path):
        img = Image.open(path).convert('RGB')
        tex = self.ctx.texture(img.size, 3, img.tobytes())
        tex.build_mipmaps()
        return tex

    def update(self, dt):
        self.transition += dt * 0.05
        if self.transition >= 1.0:
            self.transition = 0.0
            self.current = self.next
            self.next = (self.next + 1) % len(self.textures)

    def bind(self, prog):
        self.textures[self.current].use(location=0)
        if len(self.textures) > 1:
            self.textures[self.next].use(location=1)
        prog['mix_factor'].value = self.transition


In [ ]:
# 🎵 Kick Detection
def detect_kick(fft_chunk):
    low = np.mean(fft_chunk[0:15])
    mids = np.mean(fft_chunk[15:40])
    return low > 5000 and mids < 1000


In [ ]:
# 🧹 Shader Loader
def load_shader(shader_name):
    with open(f"shaders/{shader_name}.vert") as f:
        vertex = f.read()
    with open(f"shaders/{shader_name}.frag") as f:
        fragment = f.read()
    return vertex, fragment


In [ ]:
# 📁 Make sure shaders are available (clone repo if missing)
if not os.path.exists("shaders"):
    print("🔻 Downloading shaders from GitHub...")
    !git clone https://github.com/jakeroseboro/ai_music_visualizer.git temp_repo
    !mv temp_repo/shaders ./shaders
    !rm -rf temp_repo
    print("✅ Shaders ready!")

In [ ]:
# 🎵 Upload audio
from google.colab import files
print("🎵 Please upload your .wav or .mp3 file...")
uploaded = files.upload()
audio_path = next(iter(uploaded.keys()))
print(f"✅ Uploaded: {audio_path}")


In [ ]:
# ✏️ Ask for shader and prompts
shader_choice = input("🌀 Choose shader style (fractal_blob / tunnel_wave / grid_glitch / storm_warp): ").strip()
prompts = input("🖼️ Enter comma-separated AI prompts for scenes (ex: 'psychedelic mushrooms, cosmic desert'): ").strip()
prompts = [p.strip() for p in prompts.split(",")]


In [ ]:
# 🚀 Main rendering
WIDTH, HEIGHT = 1280, 720
FPS = 30

pygame.init()
pygame.display.gl_set_attribute(pygame.GL_CONTEXT_MAJOR_VERSION, 3)
pygame.display.gl_set_attribute(pygame.GL_CONTEXT_MINOR_VERSION, 3)
pygame.display.gl_set_attribute(pygame.GL_CONTEXT_PROFILE_MASK, pygame.GL_CONTEXT_PROFILE_CORE)
screen = pygame.display.set_mode((WIDTH, HEIGHT), pygame.OPENGL | pygame.DOUBLEBUF)
ctx = moderngl.create_context()
ctx.viewport = (0, 0, WIDTH, HEIGHT)

vertex_shader, fragment_shader = load_shader(shader_choice)
prog = ctx.program(vertex_shader=vertex_shader, fragment_shader=fragment_shader)

vertices = np.array([
    -1, -1,  0, 0,
     1, -1,  1, 0,
    -1,  1,  0, 1,
    -1,  1,  0, 1,
     1, -1,  1, 0,
     1,  1,  1, 1
], dtype='f4')
vbo = ctx.buffer(vertices.tobytes())
vao = ctx.simple_vertex_array(prog, vbo, 'in_vert', 'in_uv')

# 🎬 Generate AI scenes
scene_gen = SceneGenerator(prompts)
scene_paths = scene_gen.generate()

pygame.mixer.init()
pygame.mixer.music.load(audio_path)
pygame.mixer.music.play()

audio_data = pygame.mixer.Sound(audio_path).get_raw()
audio_array = np.frombuffer(audio_data, dtype=np.int16)
audio_pos = 0
chunk_size = 1024

fader = SceneFader(ctx, scene_paths)

time_val = 0.0
clock = pygame.time.Clock()
frames = []

while pygame.mixer.music.get_busy():
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            pygame.quit()
            sys.exit()

    if audio_pos + chunk_size < len(audio_array):
        chunk = audio_array[audio_pos:audio_pos+chunk_size]
        fft = np.abs(np.fft.fft(chunk)[:chunk_size // 2])
        bass = np.mean(fft[0:20]) / 5000.0
        kick = 1.0 if detect_kick(fft) else 0.0
        audio_pos += chunk_size
    else:
        bass, kick = 0.0, 0.0

    fader.update(clock.get_time() / 1000.0)

    if 'iTime' in prog:
        prog['iTime'].value = time_val
    if 'iResolution' in prog:
        prog['iResolution'].value = (WIDTH, HEIGHT)
    if 'iBass' in prog:
        prog['iBass'].value = bass
    if 'kick' in prog:
        prog['kick'].value = kick
    if 'mix_factor' in prog:
        fader.bind(prog)

    ctx.clear()
    vao.render()

    frame = pygame.image.tostring(screen, "RGB")
    frames.append(frame)

    pygame.display.flip()
    time_val += clock.get_time() / 1000.0
    clock.tick(FPS)


In [ ]:
# 📹 Save final output
writer = imageio.get_writer('output_video.mp4', fps=FPS)
for frame in frames:
    img = np.frombuffer(frame, dtype=np.uint8).reshape((HEIGHT, WIDTH, 3))
    writer.append_data(img)
writer.close()

video_clip = VideoFileClip("output_video.mp4")
audio_clip = AudioFileClip(audio_path)
final_clip = video_clip.set_audio(audio_clip)
final_clip.write_videofile("final_output.mp4", codec='libx264')

pygame.quit()
print("✅ Done! Final video: final_output.mp4")
